In [29]:
#NB!https://github.com/estnltk/estnltk/blob/master/README.md 
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import shutil
import os

In [30]:
#Katsetatud 3 tabeliga:
#Pohitabel_stopid_eemald_lower
#Pohitabel_ainult_reso
#Pohitabel_ilma_resota
data = pd.read_csv("Pohitabel_ilma_resota.csv")
data

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud
0,C01PointnumeroteAltN,"27 Kõigepealt väärib märkimist, et väljakujunenud kohtupraktika kohaselt on ELTL artiklis 267 sätestatud liikmesriigi kohtute ja Euroopa Kohtu vahelises koostöömenetluses Euroopa Kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus, mis võimaldaks viimasel poolelioleva kohtuasja lahendada. Seda arvestades tuleb Euroopa Kohtul temale esitatud küsimused vajaduse korral ümber sõnastada (2. aprilli 2020. aasta kohtuotsus Ruska Federacija, C‑897/19 PPU, EU:C:2020:262, punkt 43 ja seal viidatud kohtupraktika).",NaN,C‑416/20 PPU,0,algus,27 väärib märkimist väljakujunenud kohtupraktika eltl artiklis 267 sätestatud liikmesriigi kohtute euroopa kohtu vahelises koostöömenetluses euroopa kohtu ülesanne anda liikmesriigi kohtule tarvilik vastus võimaldaks viimasel poolelioleva kohtuasja lahendada arvestades euroopa kohtul esitatud küsimused vajaduse sõnastada 2. aprilli 2020. aasta kohtuotsus ruska federacija c‑897/19 ppu eu c:2020:262 punkt 43 viidatud kohtupraktika
1,C01PointnumeroteAltN,"28 Käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas TRi üleandmine Rumeenia ametiasutustele on õiguspärane seaduse rahvusvahelise õigusabi kohta kriminaalasjades § 83 sätete alusel, millega rakendatakse raamotsuse 2002/584 artiklit 4a Saksa õiguses.",NaN,C‑416/20 PPU,0,algus,28 käesoleval nähtub eelotsusetaotlusest eelotsusetaotluse esitanud kohus otsuse tri üleandmine rumeenia ametiasutustele õiguspärane seaduse rahvusvahelise õigusabi kriminaalasjades § 83 sätete alusel rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses
2,C01PointnumeroteAltN,"29 Eelotsusetaotluse esitanud kohtu hinnangul on üleandmiseks vajalikud tingimused täidetud, kuna esiteks on see isik teadlikult hoidunud menetlustest – mille tulemusel on tehtud need Euroopa vahistamismäärused, mida see kohus peab täitma –, põgenedes Saksamaale ja takistades seega talle kohtukutse isiklikult kättetoimetamist, ning teiseks esindas seda isikut esimeses kohtuastmes tema valitud advokaat ja apellatsiooniastmes kohtu määratud kaitsja. TR väitis aga selles kohtus, et see üleandmine ei ole direktiivi 2016/343 artiklites 8 ja 9 sätestatud nõudeid arvestades õiguspärane, kuna Rumeenias ei ole mingit tagatist tema suhtes toimunud kriminaalmenetluse uuendamiseks.",NaN,C‑416/20 PPU,0,algus,29 eelotsusetaotluse esitanud kohtu hinnangul üleandmiseks vajalikud tingimused täidetud esiteks isik teadlikult hoidunud menetlustest – tulemusel euroopa vahistamismäärused kohus täitma – põgenedes saksamaale takistades kohtukutse isiklikult kättetoimetamist esindas isikut esimeses kohtuastmes valitud advokaat apellatsiooniastmes kohtu määratud kaitsja tr väitis kohtus üleandmine direktiivi 2016/343 artiklites 8 9 sätestatud nõudeid arvestades õiguspärane rumeenias tagatist toimunud kriminaalmenetluse uuendamiseks
3,C01PointnumeroteAltN,"30 Neil asjaoludel tuleb mõista eelotsusetaotluse esitanud kohtu küsimust nii, et see puudutab küsimust, kas raamotsuse 2002/584 artiklit 4a tuleb tõlgendada nii, et vahistamismäärust täitev õigusasutus võib keelduda Euroopa vahistamismääruse täitmisest, mis on tehtud vabadusekaotuse või vabadust piirava julgeolekumeetme täitmiseks, kui asjaomane isik on takistanud talle kohtukutse isiklikult kättetoimetamist ning pole ilmunud isiklikult kohtulikule arutelule, kuna ta oli põgenenud vahistamismäärust täitvasse riiki, vaid seetõttu, et tal pole tagatist, et tema üleandmise korral vahistamismääruse teinud liikmesriigile on tal õigus uuele kohtulikule arutelule, mis on määratletud direktiivi 2016/343 artiklites 8 ja 9.",NaN,C‑416/20 PPU,0,algus,30 asjaoludel mõista eelotsusetaotluse esitanud kohtu küsimust puudutab küsimust raamotsuse 2002/584 artiklit 4a tõlgendada vahistamismäärust täitev õigusasutus keelduda euroopa vahistamismääruse täitmisest vabadusekaotuse vabadust piirava julgeolekumeetme täitmiseks asj

In [31]:
#paneme listi stop_eemaldatud
puhas=data[~pd.isnull(data.stop_eemaldatud)] #eemaldame enne ära null väärtused
column_list=puhas['stop_eemaldatud'].to_list()

In [32]:
# https://colab.research.google.com/drive/1Uh-w_oCllFMaUQOBZVL6xsdSyOfXBTxK?usp=sharing#scrollTo=AUORCu_ztPwG 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

In [33]:
#Meetod teksti sõnestamiseks ja lemmatiseerimiseks estnltk abil
from estnltk import Text
def lemmatize_with_estnltk(input_string):
    lemmas = []
    text = Text(input_string).tag_layer()
    for word in text.morph_analysis:
        lemmas.append(word.lemma[0])
    return lemmas

vectorizer = TfidfVectorizer(tokenizer = lemmatize_with_estnltk)

In [34]:
# Andmete viimine vektorkujule
matrix = vectorizer.fit_transform(column_list)
matrix.shape

(295235, 101413)

In [35]:
#salvestame matrixi maha, et ei peaks iga kord uuesti laadima
#https://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format
#from scipy import sparse

#sparse.save_npz("countvec_matrix.npz", matrix)

#uuesti avamiseks
#your_matrix_back = sparse.load_npz("countvec_matrix.npz")

In [36]:
#loodud proovilause, millega testida sarnasusi
listike=['käibemaksukohustuslaseks registreerimine tinkivinki']

In [37]:
# Proovilause viimine vektorkujule
vordluslist = vectorizer.transform(listike)
vordluslist.shape

(1, 101413)

In [38]:
#Leian koossiinuse sarnasused
#Leian esimese 200 sarnase indeksid
#Leian esimese 200 koossiinuse sarnasused, mis vastavad indeksitele
#ühendan indeksid ja koosiinuse sarnasused ühte tabelisse
sarnased_maatriks=cosine_similarity(matrix, vordluslist)
sorteeritud_indeksid=np.argsort(np.squeeze(sarnased_maatriks))
listik_indx=[]
for i in range(1,201):
    listik_indx.append(sorteeritud_indeksid[len(sorteeritud_indeksid) - i])
koos_dist=sarnased_maatriks[listik_indx]
koos_dist_jarj=[]
for i in koos_dist:
    koos_dist_jarj.append(i[0])      
print(koos_dist_jarj)
print(listik_indx)
tabel_sarnasus = pd.DataFrame(columns=['Kohtuasja_indx', 'Koos_sarnasus', 'Jarjekord'])
tabel_sarnasus['Kohtuasja_indx'] = listik_indx
tabel_sarnasus['Koos_sarnasus'] = koos_dist_jarj
tabel_sarnasus['Jarjekord'] = list(range(1, 201))
Tabel_sarnased=tabel_sarnasus.set_index('Kohtuasja_indx') 
#tabel_sarnasus
Tabel_sarnased

[0.5720260021904678, 0.5482431369391864, 0.5357503303404298, 0.5340043887336998, 0.53152668195503, 0.5093649617996797, 0.5036779228400489, 0.47624151959247124, 0.4744125736487369, 0.4731689474679306, 0.46857007437605713, 0.4653583973673614, 0.4517919416110812, 0.4372810204317653, 0.41897020241239735, 0.4187334069621734, 0.41516256014190633, 0.40964219338020535, 0.40920752947354055, 0.4038866875446101, 0.40303863094256887, 0.40049851537522574, 0.3998813358926051, 0.3959863021994201, 0.3947375021895213, 0.38728009823768295, 0.3854774896377661, 0.3835313537042385, 0.3835155550004976, 0.38267786721996466, 0.3798674194651773, 0.3789107893719772, 0.36933176087317365, 0.36793078457566986, 0.36701418648881323, 0.36518576419950766, 0.36399359834800693, 0.36029481551939024, 0.35828160220329375, 0.35401082169736037, 0.3532648362166629, 0.35303913627297295, 0.3528720390778445, 0.35241640831569476, 0.34856849258832745, 0.3483932543651536, 0.3474891437732192, 0.34648966091540634, 0.34067865010552145

,Koos_sarnasus,Jarjekord
Kohtuasja_indx,,
43850,0.572026,1
108432,0.548243,2
108438,0.535750,3
108427,0.534004,4
108425,0.531527,5
...,...,...
43841,0.224391,196
69557,0.224016,197
112791,0.223429,198


In [39]:
#Filtreerin indeksite põhjal välja sarnased lõigud
filtreeritud_sarnased=data[data.index.isin(listik_indx)]
filtreeritud_sarnased

,Tag,Lõigu sisu,Pealkiri1,Kohtuasja_nr,Kohtuasja_ID,algus_lopp,stop_eemaldatud
3015,C01PointnumeroteAltN,"29 Seoses mõistega „käibemaksukohustuslane“ – kuigi näib, et eelotsusetaotluse esitanud kohus ei sea kahtluse alla, et selline korteriomanike ühisus nagu WEG Tevesstraße on käibemaksukohustuslane – tuleb korrata, et käibemaksudirektiivi artikli 9 lõike 1 kohaselt on „maksukohustuslane“ „iga isik, kes mis tahes paigas teostab iseseisvalt mis tahes majandustegevust, olenemata nimetatud tegevuse eesmärgist või tulemustest“. Euroopa Kohtu praktika kohaselt annab käibemaksudirektiivi artikli 9 lõikes 1 kasutatud sõnastus, eelkõige sõnad „iga isik“, mõistele „maksukohustuslane“ laia tõlgenduse, mis on keskendunud majandustegevusega tegelemisel iseseisvusele selles tähenduses, et käibemaksukohustuslasena käsitatakse iga isikut, nii füüsilist kui ka juriidilist isikut, nii avalik‑ kui ka eraõiguslikku isikut, samuti üksusi, mis ei ole juriidilised isikud, kes objektiivselt vastavad selles sättes toodud kriteeriumidele (vt selle kohta 12. oktoobri 2016. aasta kohtuotsus Nigl jt, C‑340/15, EU:C:2016:764, punkt 27 ja seal viidatud kohtupraktika).",NaN,C‑449/19,118,algus,29 seoses mõistega „ käibemaksukohustuslane “ – näib eelotsusetaotluse esitanud kohus sea kahtluse korteriomanike ühisus weg tevesstraße käibemaksukohustuslane – korrata käibemaksudirektiivi artikli 9 lõike 1 „ maksukohustuslane “ „ isik tahes paigas teostab iseseisvalt tahes majandustegevust olenemata nimetatud tegevuse eesmärgist tulemustest “ euroopa kohtu praktika annab käibemaksudirektiivi artikli 9 lõikes 1 kasutatud sõnastus sõnad „ isik “ mõistele „ maksukohustuslane “ laia tõlgenduse keskendunud majandustegevusega tegelemisel iseseisvusele tähenduses käibemaksukohustuslasena käsitatakse isikut füüsilist juriidilist isikut avalik‑ eraõiguslikku isikut üksusi juriidilised isikud objektiivselt vastavad sättes toodud kriteeriumidele vt 12. oktoobri 2016. aasta kohtuotsus nigl jt c‑340/15 eu c:2016:764 punkt 27 viidatud kohtupraktika
4289,C01PointnumeroteAltN,"20 Nende kahe küsimusega, mida tuleb analüüsida koos, soovib eelotsusetaotluse esitanud kohus sisuliselt teada, kas direktiivi 2006/112 artikliga 90 on vastuolus riigisisesed õigusnormid, mille kohaselt on käibemaksuga maksustatava väärtuse vähendamine seatud sõltuvusse tingimusest, et kauba tarnimise või teenuse osutamise päeva ja selle maksuvähenduse kasutamiseks maksudeklaratsiooni paranduse esitamisele eelneva päeva seisuga peab võlgnik olema registreeritud käibemaksukohustuslasena ning tema suhtes ei või olla algatatud pankroti- ega likvideerimismenetlust ning selle maksuvähenduse kasutamiseks maksudeklaratsiooni paranduse esitamisele eelneva päeva seisuga peab võlausaldaja ise endiselt olema registreeritud käibemaksukohustuslasena.",NaN,C‑335/19,172,algus,20 kahe küsimusega analüüsida soovib eelotsusetaotluse esitanud kohus sisuliselt teada direktiivi 2006/112 artikliga 90 vastuolus riigisisesed õigusnormid käibemaksuga maksustatava väärtuse vähendamine seatud sõltuvusse tingimusest kauba tarnimise teenuse osutamise päeva maksuvähenduse kasutamiseks maksudeklaratsiooni paranduse esitamisele eelneva päeva seisuga võlgnik registreeritud käibemaksukohustuslasena algatatud pankroti- likvideerimismenetlust maksuvähenduse kasutamiseks maksudeklaratsiooni paranduse esitamisele eelneva päeva seisuga võlausaldaja endiselt registreeritud käibemaksukohustuslasena
4302,C01PointnumeroteAltN,"33 Esiteks, mis puudutab tingimust, mille kohaselt on käibemaksuga maksustatava väärtuse vähendamine seatud sõltuvusse asjaolust, et võlgnik on kauba tarnimise või teenuste osutamise ajal registreeritud käibemaksukohustuslasena, siis tuleb tõdeda, et sellist tingimust ei saa põhjendada vajadusega võtta arvesse ebakindlust selles osas, kas tasumata jätmine on lõplik. Asjaolu, kas võlgnik on kauba tarnimise või teenuste osutamise päeval käibemaksukohustuslane või mitte, ei võimalda nimelt iseenesest järeldada, et nõuet ei ole või

In [40]:
# Ühendan kaks tabelit ja filtreerin välja Koos_sarnasus,	Jarjekord, Lõigu sisu, Kohtuasja_nr
Tulemus = pd.concat([Tabel_sarnased, filtreeritud_sarnased], axis=1)
(Tulemus[['Koos_sarnasus', 'Jarjekord','Lõigu sisu', 'Kohtuasja_nr']]).sort_values('Jarjekord', ascending=True)

,Koos_sarnasus,Jarjekord,Lõigu sisu,Kohtuasja_nr
43850,0.572026,1,"38 Sellest tuleneb, et kui soetaja on käibemaksukohustuslasena registreeritud mitmes liikmesriigis, siis tuleb selle hindamisel, kas käibemaksudirektiivi artikli 141 punktis c ette nähtud tingimus on täidetud, arvesse võtta üksnes ühendusesisesel soetamisel kasutatud käibemaksukohustuslasena registreerimise numbrit.",C‑580/16
108432,0.548243,2,"27 Seega ei luba direktiiv 2006/112 – eelkõige selle artiklid 213 ja 214 – liikmesriigi maksuametil keelduda taotlejale käibemaksukohustuslasena registreerimise numbrit andmast vaid sel põhjusel, et ta ei suuda tõendada, et tal on käibemaksukohustuslasena registreerimise avalduse esitamise ajal olemas deklareeritud majandustegevuseks vajalikud majanduslikud, tehnilised ja rahalised vahendid.",C‑527/11
108438,0.535750,3,"33 Sellest kohtupraktikast järeldub, et käibemaksukohustuslasena registreerimine on selline vorminõue, mis ei võimalda maksukohustuslast takistada teostamast talle kuuluvat mahaarvamisõigust põhjusel, et ta ei olnud enne soetatud kaupade oma maksustatavas majandustegevuses kasutamist käibemaksukohustuslasena registreeritud (vt selle kohta eespool viidatud kohtuotsused Nidera Handelscompagnie, punkt 51, ja Dankowski, punktid 33, 34 ja 36). Siit tulenevalt ei saa käibemaksukohustuslasena registreerimise numbri andmisest keeldumine põhimõtteliselt avaldada mingit mõju maksukohustuslase õigusele arvata maha sisendkäibemaks, kui selle õiguse tekkimise sisulised tingimused on täidetud.",C‑527/11
108427,0.534004,4,"22 Tuleb nentida, et kuigi direktiivi 2006/112 artiklis 214 on loetletud isikute kategooriad, kes peavad olema individuaalse numbriga registreeritud, ei näe see säte ette, milliseid tingimusi võib käibemaksukohustuslasena registreerimise numbri andmisele seada. Selle artikli ja sama direktiivi artikli 213 sõnastusest nähtub, et liikmesriikidel on tegelikult käibemaksukohustuslaste registreerimise tagamise meetmete võtmisel teatav kaalutlusruum.",C‑527/11
108425,0.531527,5,"20 Peale selle on käibemaksukohustuslasena registreerimise numbril oluline tähtsus tehtud tehingute tõendamisel. Nimelt on direktiivis 2006/112 rida sätteid, mis puudutavad eeskätt arve esitamist, deklaratsioone ja koondaruandeid ning muu hulgas nõuavad, et neile dokumentidele peab olema märgitud maksukohustuslase, kauba soetaja või teenuste saaja käibemaksukohustuslasena registreerimise number.",C‑527/11
...,...,...,...,...
43841,0.224391,196,"29 Nende tingimuste hulgas nõuab käibemaksudirektiivi artikli 141 punkt a, et kaubad soetaks maksukohustuslane, kelle registrijärgne asukoht ei ole ühendusesisese lähetamise või vedamise sihtliikmesriigis, kuid kes on käibemaksukohustuslasena registreeritud teises liikmesriigis.",C‑580/16
69557,0.224016,197,"59 Eeltoodut arvestades tuleb esitatud küsimusele vastata, et kuuenda direktiivi artikli 22 lõiget 8 direktiivi artiklist 28h tulenevas redaktsioonis ning selle direktiivi artikli 28c A‑osa punkti a esimest lõiku ja punkti d tuleb tõlgendada nii, et nendega on vastuolus, kui päritoluliikmesriigi maksuhaldur keeldub käibemaksust vabastamast ühendusesisest teise liikmesriiki toimetamist põhjendusel, et maksukohustuslane ei teatanud sihtliikmesriigis antud käibemaksukohustuslasena registreerimise numbrit, kui puudub konkreetne alus eeldada maksudest kõrvalehoidumist, kaup toimetati teise liikmesriiki ja teisedki käibemaksust vabastamise tingimused on täidetud.",C‑24/15
112791,0.223429,198,"48 Esiteks, mis puudutab rikkumise raskust, mille eest selle sanktsiooniga karistatakse, siis tuleb rõhutada, et selle eesmärk oli karistada ainult käibemaksukohustuslasena registreerimise kohustuse rikkumise eest. Euroopa Kohus on sellega seoses täpsustanud, et käibemaksudirektiivi artiklis 213 sätestatud kohustused, sh maksukohustuslase kohustus deklareerida, millal tema tegevus maksukohustuslasena algab, kujutab endast ainult järelevalve eesmärgil kehtestatud vorminõuet (vt selle kohta 21. oktoobri 2010

In [41]:
# Tulemuse salvestamine - 3 tabelit
#Count_vec_tulemus
#Count_vec_tulemus_ainult_reso
#Count_vec_tulemus_ilma_resota
Tulemus.to_csv("Count_vec_tulemus_ilma_resota.csv", index=False)